# 실시간 이상치 감지 (라즈베리파이 내 구현)


### 문제는 실시간으로 잡아내는 것을 언제 시작하고, 언제 계산해서 끝낼 지
- 문제는 시작 시점을 시작할 수 있는가? --> 픽업완료부터 측정 / 배달완료까지 측정
- 그럼 배달완료에서 측정이 종료될 때 데이터베이스로 넘어가도록 (이거는 주호나 재환이가 하겠지?)
- 그럼 빅데이터는? -> count만 되도록 만들어 두자

In [2]:
# 슈방 한 번 해봅시다.
# 이거는 예제 파일로 한 코드 구현입니당!

# 일단 정규화용 평균찾기 ==> 기본값은 데이터를 수집한 것을 계속해서 받아다가 전체 평균을 내봅시다.
X_mean = -14.06441677067501
X_std = 33.97611978794694
Y_mean = 17.314758936075098
Y_std = 22.79429532634784
Z_mean = -109.86382844702727
Z_std = 85.63936548508705


# 나중에 라즈베리파이에 적용할 때는 함수형태가 아닌 그냥 코드로 집어넣으면 된다!!
def X_detect_outlier(sample1, sample2): 
    
    count_x = 0
    
    # sample1은 직전값  / sample2는 현재값
    pre_time = (sample1 - X_mean) / X_std
    real_time = (sample2 - X_mean) / X_std
    
    detect = abs(real_time - pre_time)
    
    if detect > 0.3 :
        print("alert")
        count_x += 1
    else :
        print("안전운전하세요.")
        
def Y_detect_outlier(sample1, sample2): 
    
    count_y = 0
    
    # sample1은 직전값  / sample2는 현재값
    pre_time = (sample1 - Y_mean) / Y_std
    real_time = (sample2 - Y_mean) / Y_std
    
    detect = abs(real_time - pre_time)
    
    if detect > 0.3 :
        print("alert")
        count_y += 1
    else :
        print("안전운전하세요.")

        
# 배달완료 후 마지막에 count를 더해야 함
count = count_x + count_y

NameError: name 'count_x' is not defined

# 주호가 카운트 하기 편하게, 아예 넣기 쉽게 만들어봅시다.
- 해당 평균과 표준편차 등의 parameter는 주호의 자전거 데이터를 바탕으로 만든 것입니당! (자전거.csv)

In [ ]:
## 이상치를 찾기 위한 변수 설정

# X와 Y의 평균과 표준편차
X_mean = -866.465565967574
X_std = 1613.4888133990205
Y_mean = -5726.632753235162
Y_std = 2903.044587371531

# X축 사분위 값 (각각 1분위, 4분위, 이상치 시작값)
X_Q1 : 0.21568169367526852
X_Q4 : 14.534962873771255
X_outlier : 2.3601031307914444
    
# Y축 사분위 값 (각각 1분위, 4분위, 이상치 시작값)
Y_Q1 : 0.17003787386015934
Y_Q4 : 11.85046713306096
Y_outlier : 1.775136769350455

###############################################################################################
## 여기서 부터는 이상치를 찾아내는 코드!!


# X 변화량과 Y 변화량이 각각 커졌을 때에도, 한 번에 alert가 뜨도록 만들어봅시다.

# 각 변화량을 체크할 카운트 
count_x = 0
count_y = 0

# sample1은 직전값  / sample2는 현재값
X_pre_time = (X_sample1 - X_mean) / X_std
X_real_time = (X_sample2 - X_mean) / X_std

detect_X_outlier = abs(X_real_time - X_pre_time)

if detect_X_outlier > X_outlier :
    print("alert")
    count_x += 1
else :
    pass


# Y_sample1은 accel_y 직전값  / Y_sample2는 accel_y 현재값
Y_pre_time = (Y_sample1 - Y_mean) / Y_std
Y_real_time = (Y_sample2 - Y_mean) / Y_std

detect_Y_outlier = abs(Y_real_time - Y_pre_time)

if detect_Y_outlier > Y_outlier :
    print("alert")
    count_y += 1
else :
    pass

        
# 배달완료 후 마지막에 count를 더해야 함
count = count_x + count_y

In [ ]:
# 이상치 제거 방법

# http://www.databaser.net/moniwiki/wiki.php/%EC%9D%B4%EC%83%81%EC%B9%98%EC%A0%9C%EA%B1%B0%EB%B0%A9%EB%B2%95


# 지금 접근방식은 매우 훌륭
# 타임테이블과 축의 변화값을 5-7개씩 묶어서 변화값대로 라벨을 줄 수 있을 것 같다 (급회전, 급정거, 유턴 등 -> 시간으로 파악)
# 하지만 지금 임계치를 설정한 것은 잘못된 것
# outlier를 찾는 방식으로 근거를 제대로 마련해보자.
# 사분위 수를 계산해서 찾아내는 방법이 있다.